# Wake county restaurant inspection project by Paige McKenzie
Implements methods discussed in related [blog post]().

Data courtesy of Wake County Open Data (pulled 7/18/19):
* [Restaurants](https://data-wake.opendata.arcgis.com/datasets/restaurants-in-wake-county)
* [Inspections](https://data-wake.opendata.arcgis.com/datasets/food-inspections)
* [Violations](https://data-wake.opendata.arcgis.com/datasets/food-inspection-violations)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Import data to Pandas dataframes

In [ ]:
rest = pd.read_csv('./data/Restaurants_in_Wake_County.csv', index_col=['OBJECTID'],
                  parse_dates=['RESTAURANTOPENDATE'], infer_datetime_format=True)

insp = pd.read_csv('./data/Food_Inspections.csv', index_col=['OBJECTID'],
                  parse_dates=['DATE_'], infer_datetime_format=True)

viol = pd.read_csv('./data/Food_Inspection_Violations.csv',
                  parse_dates=['INSPECTDATE'], infer_datetime_format=True, low_memory=False)

## General data exploration and cleaning
### Restaurants dataset

In [ ]:
rest.head(2)

In [ ]:
pd.concat([rest.dtypes.rename("Datatype"),
           rest.apply(pd.Series.nunique).rename("# of unique values"),
            rest.apply(pd.Series.isnull).mean().rename("% of missing values")],
          axis=1, sort=True)

In [ ]:
# city is not standardized - lowercase, replace hyphen with space
rest['CITY'] = rest['CITY'].str.lower().str.replace('-', ' ')

# combine any 'CITY' value with less than 10 data points into an "other" category, treat nulls the same
rest.loc[rest['CITY'].isin(rest['CITY'].value_counts()[rest['CITY'].value_counts()<10].index),
        'CITY'] = 'other'
rest['CITY'].fillna('other', inplace=True)

In [ ]:
# postal code isn't standardized either - abbreviate to 5 digit format, treat as integer
rest['POSTALCODE'] = rest['POSTALCODE'].apply(lambda st:int(st[:5]))

In [ ]:
from re import findall

# clean name by removing odd characters, lowercase (combining weird apostrophes)
rest['NAME'] = rest['NAME'].str.lower().str.replace('`', "'").apply(lambda x:' '.join(findall(r"([a-z'-]+)(?=\s|$)", x)))

In [ ]:
# there is no reason to retain the STATE field - it has no information, and all of Wake County is in NC anyway
rest.drop('STATE', axis=1, inplace=True)

In [ ]:
pd.concat([rest.dtypes.rename("Datatype"),
           rest.apply(pd.Series.nunique).rename("# of unique values"),
            rest.apply(pd.Series.isnull).mean().rename("% of missing values")],
          axis=1, sort=True)

### Inspections dataset

In [ ]:
# rename date column to match other table
insp.rename(columns={'DATE_':'INSPECTDATE'}, inplace=True)
insp.head(2)

In [ ]:
insp['HSISID'].isin(rest['HSISID']).mean()

We need to be able to connect inspections to restaurants, so we'll only retain rows where this is possible, about 97% of this dataset.

In [ ]:
insp = insp[insp['HSISID'].isin(rest['HSISID'])]

In [ ]:
pd.concat([insp.dtypes.rename("Datatype"),
           insp.apply(pd.Series.nunique).rename("# of unique values"),
            insp.apply(pd.Series.isnull).mean().rename("% of missing values")],
          axis=1, sort=True)

### Violations dataset

In [ ]:
viol.head(2)

In [ ]:
pd.concat([viol.dtypes.rename("Datatype"),
           viol.apply(pd.Series.nunique).rename("# of unique values"),
            viol.apply(pd.Series.isnull).mean().rename("% of missing values")],
          axis=1, sort=True)

The data came without HSISID populated, but we may be able to crosswalk with the other tables to get this information. However, it is worth noting that the `violations` table has dramatically more unique `PERMITID` values than the other tables, meaning it will be impossible to connect these violations to permits/restaurants in the dataset.

In [ ]:
viol['PERMITID'].isin(insp['PERMITID']).mean()

In fact, it looks like about 17% of the rows in the `violations` table can be connected to the others, so this is the subset we'll keep.

In [ ]:
# check there is one HSISID for each PERMITID
assert (rest.groupby('PERMITID')['HSISID'].nunique()>1).sum()==0

viol = viol.drop('HSISID', axis=1).join(insp.set_index('PERMITID')['HSISID'].drop_duplicates(),
         on='PERMITID', how='inner')

In [ ]:
pd.concat([viol.dtypes.rename("Datatype"),
           viol.apply(pd.Series.nunique).rename("# of unique values"),
            viol.apply(pd.Series.isnull).mean().rename("% of missing values")],
          axis=1, sort=True)

We've lost data, but at least everything that remains is usable.

### Visualizations

In [ ]:
print(rest.index.nunique(), "restaurants")
print(len(viol), "violations")
print(len(insp[['HSISID', 'INSPECTDATE']].drop_duplicates()), "inspectors")
print(insp['INSPECTOR'].nunique(), "inspectors")

In [ ]:
print("Date range:\n{}".format(insp['INSPECTDATE'].agg([min, max])))

In [ ]:
data = insp.groupby(pd.Grouper(freq="M", key='INSPECTDATE')).size()
plt.plot(data, color='#31394d')
plt.ylim((0,800))
plt.title("Number of inspections conducted, per month")
plt.ylabel("# of inspections")
plt.xlabel("Date")

# calc the trendline
z = np.polyfit(list(range(len(data))), data.values, 1)
p = np.poly1d(z)
plt.plot(data.index, p(list(range(len(data)))), color='#009384', linestyle='--')
# the line equation:
plt.figtext(x=.14, y=.14, s="y = %.3f * Months + %.3f"%(z[0],z[1]), color='#009384')

plt.show()

In [ ]:
data = viol.groupby([pd.Grouper(freq="M", key='INSPECTDATE'), viol['PERMITID']]).size().rename("VIOLCOUNT").groupby(level=0).mean()
plt.plot(data, color='#31394d')
plt.ylim((0,12))
plt.title("Average number of violations found in\nan inspection, per month")
plt.ylabel("# of violations")
plt.xlabel("Date")

# calc the trendline
z = np.polyfit(list(range(len(data))), data.values, 1)
p = np.poly1d(z)
plt.plot(data.index, p(list(range(len(data)))), color='#009384', linestyle='--')
# the line equation:
plt.figtext(x=.14, y=.14, s="y = %.3f * Months + %.3f"%(z[0],z[1]), color='#009384')

plt.show()

In [ ]:
insp.groupby('HSISID').size().div(insp.groupby('HSISID')['INSPECTDATE'].apply(lambda s:(s.max()-s.min()).days/365
                                                                             if len(s)>1 else None)).dropna().hist(bins=30, color='#31394d')
plt.title("Histogram of restaurants' # of inspections/year")
plt.show()

## Goal
Imagine it's Christmas break, end of 2018, and we've scheduled restaurant inspections for the first 6 months of 2019. In hopes of increasing the scores of at-risk restaurants, we'd like to implement a mailing campaign, targeted at restaurants who we expect to score low, giving them information about common violations and advice for proper procedures. Hopefully, this will elicit improvements before our inspector visits, and they will receive a higher inspection `SCORE`. 

Due to budget constraints, we can only mail to 500 restaurants in this campaign. We want to target the most "at-risk" customers using data science. For this, we want a model that can accurately predict which inspection is likely to receive a low `SCORE`, so we can target them.

We define a "low" `SCORE` as `SCORE`<93.

In [ ]:
validation_date = '2019-01-01'
test_date = '2018-07-01'

**Approach:**

We will first engineer a set of features from what we know about each restaurant. This set will be split into a train and test set using a 70/30 split. Using appropriate models and grid search to select optimum parameters, we'll identify the best model using cross validation within the training set, and then the model that performs best on the reserved testing set will be selected for production.

**Feature engineering:**

Features will include:
* the number of days the restaurant has been open (`TIMEOPEN`, integer)
* the number of days since the restaurant was last inspected (`TIMESINCE`, integer)
* the number of other restaurants (unique HSISIDs) with the same name (`CHAINCOUNT`, integer)
* the number of inspections for the restaurant (`INSPCOUNT`, integer)
* whether the restaurant has ever needed a re-inspection (`WASREINSP`, binary)
* the average number of violations per inspection for that restaurant (`AVGVIOL`, float)

Just for fun and in case of some seasonality to inspections, we'll throw in cyclical features for month:
* `SINMONTH` (float)
* `COSMONTH` (float)

Including preexisting features:
* `FACILITYTYPE` (categorical and will be converted to dummy variables)
* `CITY` (categorical and will be converted to dummy variables)
* `POSTALCODE` (integer, due to the fact that zipcodes that are close in number tend to be geographically similar as well)
* `AREACODE` (categorical, will be extracted from the existing `PHONE NUMBER` field

Wrapping in a few details on the restaurant's most recent inspection, prior to that in question:
* the type of the restaurant's most recent inspection (`TYPE_previous`, categorical to dummy)
* the inspector who completed the most recent inspection (`INSPECTOR_previous`, categorical to dummy)
* the previous inspection score (`SCORE_previous`, float)

In [ ]:
def engineer_features(insp, rest):
    features = insp[['HSISID', 'INSPECTDATE', 'SCORE', 'INSPECTOR', 'TYPE']].join(
        rest.set_index('HSISID')[['NAME', 'CITY', 'RESTAURANTOPENDATE', 'FACILITYTYPE', 'PHONENUMBER']], on='HSISID', how='left')

    # calculate number of unique HSISID's with the same name (# of chain locations, maybe)
    features = features.join((features.groupby('NAME')['HSISID'].nunique()).rename('CHAINCOUNT'), on='NAME')

    # calculate number of inspections
    features = features.join(insp.groupby('HSISID').size().rename("INSPCOUNT").reindex(features['HSISID'].unique(), fill_value=0), on='HSISID')

    # whether the restaurant ever needed a re-inspection
    features['WASREINSP'] = features['HSISID'].isin(insp.loc[insp['TYPE']=='Re-Inspection', 'HSISID'].unique()).astype(int)

    # average number of violations per inspection
    features = features.join(insp.groupby(['HSISID', 'INSPECTDATE']).size().reindex(features[['HSISID', 'INSPECTDATE']], fill_value=0).mean(level=0).rename("AVGVIOL"),
                            on='HSISID')

    # self-merge to get information about previous inspection
    features = features.groupby('HSISID')[['SCORE', 'INSPECTOR', 'TYPE', 'INSPECTDATE']].apply(lambda group:
                 pd.concat([group.add_suffix('_previous'),
                            group.shift(-1).add_suffix('_current')],
                           axis=1, sort=True)).drop(['INSPECTOR_current', 'TYPE_current'], axis=1).\
    rename(columns={'INSPECTDATE_current':'INSPECTDATE'}).\
    merge(features, on=['HSISID', 'INSPECTDATE'], how='right').drop_duplicates()


    # time features
    features['TIMEOPEN'] = (features['INSPECTDATE'] - features['RESTAURANTOPENDATE']).dt.days
    features.loc[features['INSPECTDATE_previous'].isnull(), 'INSPECTDATE_previous'] = features['RESTAURANTOPENDATE']
    features['TIMESINCE'] = (features['INSPECTDATE'] - features['INSPECTDATE_previous']).dt.days
    from numpy import sin, cos
    from math import pi
    features['SINMONTH'] = features['INSPECTDATE'].dt.month.apply(lambda x:sin(2*pi*x/12))
    features['COSMONTH'] = features['INSPECTDATE'].dt.month.apply(lambda x:cos(2*pi*x/12))

    # derive area code feature as dummy columns
    features = pd.concat([features, 
                          pd.get_dummies(features['PHONENUMBER'].str[1:4])],
                         axis=1)

    # drop extra columns
    features = features.drop(['INSPECTOR', 'TYPE', 'RESTAURANTOPENDATE', 'INSPECTDATE_previous', 'SCORE_current', 'PHONENUMBER'], 
                             axis=1)
    
    # inpute missing values
    features.loc[features['INSPECTOR_previous'].isnull(), 'INSPECTOR_previous'] = 'none' # simply add a new category
    features.loc[features['TYPE_previous'].isnull(), 'TYPE_previous'] = 'none' # simply add a new category
    features['NAME'].fillna('', inplace=True)
    features['CHAINCOUNT'].fillna(1, inplace=True)
    # fill previous score with moving average
    features.loc[features['SCORE_previous'].isnull(), 'SCORE_previous'] = pd.merge_asof(features.sort_values('INSPECTDATE'), features.groupby(pd.Grouper(freq="M", key='INSPECTDATE'))['SCORE'].mean().shift().to_frame().add_suffix('_fill'),
              left_on='INSPECTDATE', right_index=True, direction='backward')['SCORE_fill']
    return features

### Time-travel
We will engineer features for training the models (all time, prior to `test_date`), testing the models (`test_date` to `validation_date`), and then see how our model would've done using the inspections after `validation_date`.

In [ ]:
X = engineer_features(insp[insp['INSPECTDATE']<test_date], rest)

validation = engineer_features(insp, rest)
validation = validation[validation['INSPECTDATE']>=validation_date]

test = engineer_features(insp[insp['INSPECTDATE']<validation_date], rest)
test = test[test['INSPECTDATE']>=test_date]

For no reason at all, we will use NLP to parse the text in the restaurant's `NAME`, in hopes of extracting some insight into the cuisine type or food style (ie. `buffet` might indicate a higher chance of failing inspection). I've arbitrarily decided to choose the top 40 words from restaurant `NAME`, using most frequent words. Note that this will be "trained" using the training time period (`X`), and then applied to the prediction sets (`test`, `validation`).

While we're at it, we should also parse the categorical fields (`FACILITYTYPE`, `INSPECTOR_previous`, `TYPE_previous`, `CITY`), again using only the options present in training for the other sets.

In [ ]:
# column size explodes from here as we add columns for words in the restaurant name, dummy columns
from sklearn.feature_extraction.text import CountVectorizer

# fit on training set
vectorizer = CountVectorizer(stop_words='english', binary=True, max_features=40)
vectorizer.fit(X['NAME'])

# apply to all sets
X = pd.concat([X.drop('NAME', axis=1), 
       pd.DataFrame(vectorizer.transform(X['NAME']).todense().astype(int),
         columns=vectorizer.vocabulary_, index=X.index)],
      axis=1)
# get dummy columns
X = pd.get_dummies(X, drop_first=False)

validation = pd.concat([validation.drop('NAME', axis=1), 
       pd.DataFrame(vectorizer.transform(validation['NAME']).todense().astype(int),
         columns=vectorizer.vocabulary_, index=validation.index)],
      axis=1)
validation = pd.get_dummies(validation, drop_first=False).reindex(X.columns, axis=1, fill_value=0)

test = pd.concat([test.drop('NAME', axis=1), 
       pd.DataFrame(vectorizer.transform(test['NAME']).todense().astype(int),
         columns=vectorizer.vocabulary_, index=test.index)],
      axis=1)
test = pd.get_dummies(test, drop_first=False).reindex(X.columns, axis=1, fill_value=0)

**Modeling:**

We'll use cross-validation on the `X` dataset with known scores, to determine optimal model parameters. The reserved `test` set will be used to choose the best model, hopefully controlling for over-fitting. Finally, we will re-fit the model using the process we identify as best, and see how we would perform on the `validation` set, had we actually implemented the model-building process to identify the 500 most "at-risk" restaurants.

In [ ]:
(X['SCORE']<93).value_counts()

Let's balance our classes, which should hopefully help the model separate the classes better. 

In [ ]:
oversample = 3 # the number of times to include each positive target row
ratio = .3 # the goal target distribution (# of positive class)

In [ ]:
X_neg = X[X['SCORE']<93]
X_pos = X[X['SCORE']>=93]

del X

In [ ]:
X = X_pos.sample(frac=oversample, replace=True).append(
    X_neg.sample(int((1/ratio-1)*oversample*len(X_pos)), replace=True))
del X_neg, X_pos

In [ ]:
(X['SCORE']<93).value_counts()

We've successfully balanced the data to 50:50.

Next, using all inspections prior to the `test_date` and after 2014, we'll implement cross-validation to choose optimal model parameters for a simple decision tree, bagging, boosting, and random forest classifiers.

In [ ]:
# ignore inspections with missing vital data
X = X[X['SCORE'].notna()&(X['INSPECTDATE'].dt.year>=2014)]

y = (X['SCORE']<93).astype(int)
X = X.drop(['SCORE', 'INSPECTDATE'], axis=1)

# also separate test, validation into X, and y
y_test = (test['SCORE']<93).astype(int)
X_test = test.drop(['SCORE', 'INSPECTDATE'], axis=1)

y_val = (validation['SCORE']<93).astype(int)
X_val = validation.drop(['SCORE', 'INSPECTDATE'], axis=1)

del test, validation

### Begin modeling

In [ ]:
# evaluate a random prediction
from sklearn.metrics import roc_auc_score

print("Train AUC:", roc_auc_score(y, np.random.choice([0,1], size=len(y))))
print("Mailing accuracy:", y_test.loc[np.random.choice(y_test.index, size=500)].mean())

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz

tree = DecisionTreeClassifier(random_state=1)
tree_grid = GridSearchCV(tree, param_grid={'max_depth':[3,5,7,12],
                                     'max_features':[.5,.8,1.]},
                   scoring='roc_auc', cv=3, n_jobs=-1, verbose=1)
tree_grid.fit(X, y)

tree = tree_grid.best_estimator_

print("Train AUC:", roc_auc_score(y, tree.predict(X)))
print("Test AUC:", roc_auc_score(y_test, tree.predict(X_test)))

tree_mailing_acc = y_test.loc[pd.Series(tree.predict_proba(X_test)[:,1], index=X_test.index).nlargest(500).index].mean()
print("Mailing accuracy:", tree_mailing_acc)
print("Mailing lift:", round(tree_mailing_acc/y_test.mean(), 2))

In [ ]:
from sklearn.ensemble import BaggingClassifier

bags = BaggingClassifier(random_state=1)
bags_grid = GridSearchCV(bags, param_grid={'n_estimators':[20,60],
                                     'max_samples':[.5,.8]},
                   scoring='roc_auc', cv=3, n_jobs=-1, 
                         verbose=1, return_train_score=True)
bags_grid.fit(X, y)

bags = bags_grid.best_estimator_

print("Train AUC:", roc_auc_score(y, bags.predict(X)))
print("Test AUC:", roc_auc_score(y_test, bags.predict(X_test)))

bags_mailing_acc = y_test.loc[pd.Series(bags.predict_proba(X_test)[:,1], index=X_test.index).nlargest(500).index].mean()
print("Mailing accuracy:", bags_mailing_acc)
print("Mailing lift:", round(bags_mailing_acc/y_test.mean(), 2))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

boost = GradientBoostingClassifier(random_state=1)
boost_grid = GridSearchCV(bags, param_grid={'n_estimators':[20,60],
                                     'max_features':[.5,.8]},
                   scoring='roc_auc', cv=3, n_jobs=-1, 
                         verbose=1, return_train_score=True)
boost_grid.fit(X, y)

boost = boost_grid.best_estimator_

print("Train AUC:", roc_auc_score(y, boost.predict(X)))
print("Test AUC:", roc_auc_score(y_test, boost.predict(X_test)))

boost_mailing_acc = y_test.loc[pd.Series(boost.predict_proba(X_test)[:,1], index=X_test.index).nlargest(500).index].mean()
print("Mailing accuracy:", boost_mailing_acc)
print("Mailing lift:", round(boost_mailing_acc/y_test.mean(), 2))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=1)
rf_grid = GridSearchCV(bags, param_grid={'n_estimators':[40,120],
                                     'max_samples':[.3,.5,.8],
                                     'max_features':[.3,.5,.8]},
                   scoring='roc_auc', cv=3, n_jobs=-1, 
                         verbose=1, return_train_score=True)
rf_grid.fit(X, y)

rf = rf_grid.best_estimator_

print("Train AUC:", roc_auc_score(y, rf.predict(X)))
print("Test AUC:", roc_auc_score(y_test, rf.predict(X_test)))

rf_mailing_acc = y_test.loc[pd.Series(rf.predict_proba(X_test)[:,1], index=X_test.index).nlargest(500).index].mean()
print("Mailing accuracy:", rf_mailing_acc)
print("Mailing lift:", round(rf_mailing_acc/y_test.mean(), 2))

**Conclusions:**

The best out-of-sample model using log loss will be used to predict on the unknown inspections.